In [ ]:
from manim import *
config.media_width = "100%"
_RI = "-v WARNING -s --progress_bar None --disable_caching Example"
_RV = "-v WARNING -qm --progress_bar None --disable_caching Example"


In [ ]:
from random import uniform

In [ ]:
class FreehandDraw(VMobject):
    def __init__(self, path: VMobject, partitions=80, range_random=0.05, **kwargs):
        self.path = path
        self.partitions = partitions
        self.range_random = range_random
        super().__init__(**kwargs)
        self.match_style(path)

    def generate_points(self):
        coords = [
            self.get_random_normal_vector_from_proportion(i / self.partitions) 
            for i in range(self.partitions)
        ]
        if self.path.is_closed(): coords.append(coords[0])
        self.set_points_smoothly(coords)

    def get_random_normal_vector_from_proportion(self, alpha, dx=0.001):
        start    = self.path.point_from_proportion( np.clip(alpha - dx, 0, 1) )
        d_vector = self.path.point_from_proportion( np.clip(alpha + dx, 0, 1) ) - start
        normal_vector = rotate_vector(d_vector / np.linalg.norm(d_vector), PI/2) 
        return start + normal_vector * self.get_random_val()

    def get_random_val(self):
        return uniform(-self.range_random, self.range_random)

In [ ]:
%%manim $_RV
class Example(Scene):
    def construct(self):
        c = Circle().scale(3)
        c_fh = FreehandDraw(c)

        self.add(c)
        self.wait()
        self.play(Transform(c, c_fh))
        self.wait()

In [ ]:
class ArrayBlock(VGroup):
    def __init__(self, elements=[], height=1, height_proportion=1, **kwargs):
        super().__init__(*[
            VGroup(self.get_block(mob, height, height_proportion), mob)
                .set(height=height)
            for mob in elements
        ], **kwargs)
        self.arrange(RIGHT,buff=0)

    def get_block(self, mob, height, height_proportion):
        block = Square().set(height=height).move_to(mob)
        block.scale(height_proportion)
        return block

In [ ]:
%%manim $_RI
class Example(Scene):
    def construct(self):
        text = Text("Hello world")
        blocks = ArrayBlock(text)
        VGroup(*[blocks[i+1][1].align_to(blocks[i][1],DOWN) for i in range(len(blocks)-1)])

        self.add(blocks)

In [ ]:
from random import choice
from itertools import cycle

In [ ]:
class LineBooks(VGroup):
    def __init__(self, 
            n=15, width=0.2, height=1, dw=0.2, dh=0.6,
            colors=[TEAL,BLUE,GREEN,PURPLE]
        ):
        super().__init__()
        colors = cycle(colors)
        margin = Rectangle(
            width=width+dw,
            height=height+dh
        )
        self.add(*[
            Rectangle(
                width=width+uniform(0,dw),
                height=height+uniform(0,dh),
                fill_opacity=1,
            fill_color=next(colors),
            stroke_width=1
            )
            for _ in range(n)
        ])
        margin.move_to(self)
        margin.align_to(self, DOWN)
        self.arrange(RIGHT,buff=0,aligned_edge=DOWN)

In [ ]:
class Shelf(VGroup):
    def __init__(self, n=3, books_kwargs={}):
        super().__init__()
        self.shelfs = VGroup(*[
            LineBooks(**books_kwargs)
            for _ in range(n)
        ])
        self.margins = VGroup(*[
            Rectangle(width=b.width, height=b.height)
            for b in self.shelfs
        ])
        max_width  = max(*[m.width for m in self.margins])
        max_height = max(*[m.height for m in self.margins])

        margin = Rectangle(width=max_width, height=max_height)
        all_shelfs = VGroup(*[
            self.get_shelf_and_margin(b,margin)
            for b in self.shelfs
        ])
        all_shelfs.arrange(DOWN,buff=0,aligned_edge=LEFT)
        self.add(*all_shelfs)

    def get_shelf_and_margin(self, shelf, margin):
        new_margin = margin.copy()
        margin.move_to(shelf)
        margin.align_to(shelf,DOWN)
        shelf.align_to(margin,choice([LEFT,RIGHT]))
        return VGroup(shelf,new_margin)

In [ ]:
%%manim $_RI
class Example(Scene):
    def construct(self):
        grp = VGroup(LineBooks(), Shelf(4)).arrange(RIGHT)
        self.add(grp)

In [ ]:
class Spring2D(VMobject):
    def __init__(
        self,
        XY_start,       # Start spring point
        XY_end,         # End spring point
        num_loops=10,   # number of loops of spring
        r=0.2,          # radius of spring loop
        **kwargs
    ):
        super().__init__(**kwargs)
        L = np.sqrt( # Length of spring
            np.square(XY_end[0] - XY_start[0]) + np.square(XY_end[1] - XY_start[1])
        )  
        theta = np.arctan2( # Angle of spring from horizontal
            XY_end[1] - XY_start[1], XY_end[0] - XY_start[0]
        )  

        T = ( # Intermediate parameter end value: Length of spring minus start and end parts
            L - 2 * r
        )  
        alpha = ( # Intermediate const: Rotation constant (determines how fast to do loops)
            np.pi * (2 * num_loops + 1) / (L - 2 * r)
        )  

        num_pts = 1000
        t = np.linspace(0, T, num_pts)  # Intermediate parameter, t
        x = np.zeros(num_pts)           # preallocating space for x and y
        y = np.zeros(num_pts)

        x = t + r * np.cos(alpha * t - np.pi) + r
        y = r * np.sin(alpha * t - np.pi)

        # multiply by rotation matrix R(theta) and add offset
        x_rot = x * np.cos(theta) - y * np.sin(theta)
        y_rot = x * np.sin(theta) + y * np.cos(theta)

        x = x_rot + XY_start[0]
        y = y_rot + XY_start[1]

        ind_pts = np.array( # need to transpose to get each point as a separate array
            [x, y, np.zeros(num_pts)]
        ).T  
        self.set_points_smoothly(ind_pts)

In [ ]:
class Example(Scene):
    def construct(self):
        d1 = Dot(LEFT*3,color=RED)
        d2 = Dot(RIGHT*3,color=BLUE)

        spring = always_redraw(
            lambda: Spring2D(d1.get_center(),d2.get_center())
        )
        self.add(d1,d2,spring)
%manim $_RI

In [ ]:
class Example(Scene):
    def construct(self):
        d1 = Dot(LEFT*3,color=RED)
        d2 = Dot(RIGHT*3,color=BLUE)

        spring = always_redraw(
            lambda: Spring2D(d1.get_center(),d2.get_center(),r=0.4)
        )
        self.add(d1,d2,spring)
        self.wait()
        self.play(d2.animate.shift(RIGHT*2),run_time=2)
        self.wait()
        self.play(d1.animate.shift(LEFT*3),run_time=2)
        self.wait()
        self.play(
            d1.animate.shift(RIGHT*3),
            d2.animate.shift(LEFT*2),
            run_time=2
        )
        self.wait()
%manim $_RV

In [ ]:
class Damper(VGroup):
    def __init__(self, start,
                       end,
                       piston_length=None,
                       out_buff_proportion=None,
                       init_buff_proportion=0.2,
                       proportion_length=None,
                       *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.main_line = Line(start, end)
        # if init_state:
        if piston_length is None:
            self.piston_length = self.main_line.get_length() * 0.1
        else:
            self.piston_length = piston_length
        if out_buff_proportion is None:
            self.out_buff = self.piston_length * init_buff_proportion
            self.desplazable_length = self.piston_length * (1 - init_buff_proportion)
            self.init_out_buff_proportion = init_buff_proportion
        else:
            self.out_buff = self.piston_length * out_buff_proportion
            self.desplazable_length = self.piston_length * (1 - out_buff_proportion)
        # self.out_buff_proportion = 
        if proportion_length is None:
            self.proportion_length = self.desplazable_length / self.main_line.get_length()
        # print(self.proportion_length)
        self.piston_height = 0.5
        self.pre_piston = Rectangle(width=self.piston_length,height=self.piston_height)
        self.piston = VMobject()
        self.piston.set_points_as_corners([
            self.pre_piston.get_corner(UL),
            self.pre_piston.get_corner(UR),
            self.pre_piston.get_corner(DR),
            self.pre_piston.get_corner(DL),
        ])
        self.piston.move_to(self.main_line)
        self.left_line = Line(self.main_line.get_start(),self.piston.get_left())
        self.right_line = Line(self.piston.get_right(),self.main_line.get_end())
        self.inner_line = Line(self.piston.get_left(),self.piston.get_left()+RIGHT*self.out_buff)
        self.piston_vertical_line = Line(
            UP*self.piston_height*0.8/2,
            DOWN*self.piston_height*0.8/2
        ).move_to(self.inner_line.get_end())
        self.add(
            self.left_line, self.inner_line, self.piston_vertical_line,
            self.piston, self.right_line
        )

    def get_piston_length(self):
        return self.piston_length

    def init_updater(self, f1, f2):
        self.init_length = self.main_line.get_length()
        self.init_buff_prop = self.init_out_buff_proportion
        def damper_update(mob):
            distance = abs(f1() - f2())[0]
            alpha = mob.init_length / distance
            out_buff_proportion = mob.init_buff_prop - min(1 - interpolate(0, 1, alpha), mob.init_buff_prop) \
                if distance > mob.init_length \
                else mob.init_buff_prop + (1-interpolate(0, 1, 1 / alpha))
            mob.become(
                Damper(f1(),f2(),piston_length=self.get_piston_length(),out_buff_proportion=out_buff_proportion)
            )
        self.add_updater(damper_update)

In [ ]:
class Example(Scene):
    def construct(self):
        d1 = Dot(LEFT*3,color=RED)
        d2 = Dot(RIGHT*3,color=BLUE)
        damper = Damper(d1.get_center(),d2.get_center())

        self.add(d1,d2,damper)
%manim $_RI

In [ ]:
class Example(Scene):
    def construct(self):
        d1 = Dot(LEFT*3,color=RED)
        d2 = Dot(RIGHT*3,color=BLUE)
        damper = Damper(d1.get_center(),d2.get_center())
        damper.init_updater(lambda: d1.get_center(), lambda: d2.get_center())

        self.add(d1,d2,damper)
        self.wait()
        self.play(d2.animate.shift(LEFT*2),run_time=2)
        self.wait()
        self.play(d1.animate.shift(RIGHT),run_time=2)
        self.wait()
        self.play(
            d1.animate.shift(LEFT),
            d2.animate.shift(RIGHT*6),
            run_time=2
        )
        self.wait()
        self.play(d2.animate.shift(LEFT*4),run_time=2)
        self.wait()
%manim $_RV